### Création du jeu ###

dans un premier temps on créer une class TicTacToe qui represente l'environement du jeu

In [2]:
import random
import numpy as np
from jupyter_server.terminal import initialize


class TicTacToeEnv:
    def __init__(self):
        self.board = [0] * 9  # 9 positions sur le plateau
        self.current_winner = None

    #reset les case du plateau
    def reset(self):
        self.board = [0] * 9
        self.current_winner = None
        return tuple(self.board)

    #indique les case encore idsponible pour jouer son tour
    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == 0]

    #pose un pion sur le plateau, renvois false si la case est deja prise et renvois true si le coup fait ganger le joueur
    def make_move(self, position, player):
        if self.board[position] == 0:
            self.board[position] = player
            if self.check_winner(position, player):
                self.current_winner = player
            return True
        return False

    #liste des combinaison gagnants
    def check_winner(self, square, player):
        # Conditions de victoire
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # lignes
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # colonnes
            [0, 4, 8], [2, 4, 6]  # diagonales
        ]
        for condition in win_conditions:
            if square in condition and all(self.board[i] == player for i in condition):
                return True
        return False

    #indique si la partie est une egalité
    def is_draw(self):
        return 0 not in self.board and self.current_winner is None
    
    #renvois l'etiat actuel du tableau, used dasn le stockage de la Q-table
    def get_state(self):
        return tuple(self.board)


    #affichage du plateau
    def render(self):
        # Affichage du plateau
        for row in [self.board[i:i + 3] for i in range(0, 9, 3)]:
            print(" | ".join([str(x) if x != 0 else " " for x in row]))


C:\Users\guill\AppData\Local\Temp\ipykernel_5804\829602291.py:3: DeprecationWarning: Terminals support has moved to `jupyter_server_terminals`
  from jupyter_server.terminal import initialize


In [3]:

def trainAiVsPreProgrammed(agent, pre_programmed_agent, env, episodes=10000, print_every=1000):
    win_qlearning, win_preprogrammed, draw = 0, 0, 0  # Stats counters

    for episode in range(1, episodes + 1):
        state = env.reset()  # Reset the environment at the start of each game
        done = False
        turn = 1 if episode % 2 == 1 else -1  # Alternate starting player

        while not done:
            if turn == 1:  # Q-learning agent's turn
                available_moves = env.available_moves()
                action = agent.choose_action(state, available_moves)
                env.make_move(action, agent.player)
                next_state = env.get_state()

                # Update Q-table based on outcome
                if env.current_winner == agent.player:
                    agent.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    win_qlearning += 1
                    done = True
                elif env.is_draw():
                    agent.update_q_table(state, action, reward=0.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    agent.update_q_table(state, action, reward=0, next_state=next_state, done=False)
                state = next_state

            else:  # PreProgrammed agent's turn
                available_moves = env.available_moves()
                action = pre_programmed_agent.choose_action(state, env)
                env.make_move(action, pre_programmed_agent.player)
                state = env.get_state()

                # Check if pre-programmed agent won or if it's a draw
                if env.current_winner == pre_programmed_agent.player:
                    agent.update_q_table(state, action, reward=-1, next_state=state, done=True)
                    win_preprogrammed += 1
                    done = True
                elif env.is_draw():
                    agent.update_q_table(state, action, reward=0.5, next_state=state, done=True)
                    draw += 1
                    done = True

            # Alternate turn
            turn *= -1

        # Decay epsilon for exploration
        agent.decay_epsilon()

        # Print statistics every `print_every` episodes
        if episode % print_every == 0:
            print(f"Episode {episode} - Wins Q-learning: {win_qlearning}, Wins PreProgrammed: {win_preprogrammed}, Draws: {draw}")
            win_qlearning, win_preprogrammed, draw = 0, 0, 0  # Reset counters

    print("Training complete")

In [17]:
def trainAiVsRandom(ai_agent, env, episodes, ai_starts):
    
    #on définie un joueur qui va jouer aléatoirement a chaque tour
    def random_player_move(available_moves):
        return random.choice(available_moves)

    win, loss, draw = 0, 0, 0  # Compteurs pour les statistiques de jeu

    for episode in range(1, episodes + 1):
        state = env.reset()  # Réinitialise l'environnement pour chaque partie
        done = False
        turn = 1 if ai_starts else -1  # Si ai_starts est True, l'IA commence ; sinon, le joueur aléatoire commence

        while not done:
            if turn == 1:  # Tour de l'agent IA
                available_moves = env.available_moves()
                action = ai_agent.choose_action(state, available_moves)
                env.make_move(action, ai_agent.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'IA
                if env.current_winner == ai_agent.player:
                    ai_agent.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    win += 1
                    done = True
                elif env.is_draw():
                    ai_agent.update_q_table(state, action, reward=0.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    ai_agent.update_q_table(state, action, reward=0, next_state=next_state, done=False)

                state = next_state
            else:  # Tour du joueur aléatoire
                available_moves = env.available_moves()
                action = random_player_move(available_moves)
                env.make_move(action, -ai_agent.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup du joueur aléatoire
                if env.current_winner == -ai_agent.player:
                    ai_agent.update_q_table(state, action, reward=-1, next_state=next_state, done=True)
                    loss += 1
                    done = True
                elif env.is_draw():
                    ai_agent.update_q_table(state, action, reward=0.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    # Pas de mise à jour pour le joueur aléatoire
                    state = next_state

            turn *= -1  # Change le tour

        # Décroît l'exploration de l'IA après chaque partie
        ai_agent.decay_epsilon()

        # Affiche les statistiques tous les 1000 épisodes pour voir l'avancé de l'IA
        if episode % 10000 == 0:
            print(f"Épisode {episode} - Victoires de l'IA: {win}, Défaites de l'IA: {loss}, Égalités: {draw}")
            win, loss, draw = 0, 0, 0  # Réinitialise les compteurs pour la prochaine série

    print("Entraînement terminé")

In [25]:
def trainAivsAi(agent1, agent2, env, episodes=10000, print_every=1000):
  
    win_agent1, win_agent2, draw = 0, 0, 0  # Compteurs pour les statistiques

    for episode in range(1, episodes + 1):
        state = env.reset()  # Réinitialise l'environnement pour chaque partie
        done = False
        turn = 1  # 1 pour agent1, -1 pour agent2

        while not done:
            if turn == 1:  # Tour de l'agent1
                available_moves = env.available_moves()
                action = agent1.choose_action(state, available_moves)
                env.make_move(action, agent1.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'agent1
                if env.current_winner == agent1.player:
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=-1, next_state=next_state, done=True)
                    win_agent1 += 1
                    done = True
                elif env.is_draw():
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=1.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    agent1.update_q_table(state, action, reward=0, next_state=next_state, done=False)
                state = next_state
            else:  # Tour de l'agent2
                available_moves = env.available_moves()
                action = agent2.choose_action(state, available_moves)
                env.make_move(action, agent2.player)
                next_state = env.get_state()

                # Vérifie le résultat après le coup de l'agent2
                if env.current_winner == agent2.player:
                    agent1.update_q_table(state, action, reward=-2, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=2, next_state=next_state, done=True)
                    win_agent2 += 1
                    done = True
                elif env.is_draw():
                    agent1.update_q_table(state, action, reward=1, next_state=next_state, done=True)
                    agent2.update_q_table(state, action, reward=1.5, next_state=next_state, done=True)
                    draw += 1
                    done = True
                else:
                    agent2.update_q_table(state, action, reward=0, next_state=next_state, done=False)
                state = next_state

            turn *= -1  # Change le tour

        # Décroît l'exploration de chaque IA après chaque partie
        agent1.decay_epsilon()
        agent2.decay_epsilon()

        # Affiche les statistiques tous les `print_every` épisodes
        if episode % print_every == 0:
            print(f"Épisode {episode} - Victoires Agent1: {win_agent1}, Victoires Agent2: {win_agent2}, Égalités: {draw}")
            win_agent1, win_agent2, draw = 0, 0, 0  # Réinitialise les compteurs pour la prochaine série

    # Affiche les statistiques finales après l'entraînement complet
    print("Entraînement terminé")


In [5]:
class PreProgrammedAgent:
    def __init__(self, player):
        self.player = player  # 1 for X, -1 for O
        self.opponent = -player  # Opposite player

    def choose_action(self, board, env):
        available_moves = env.available_moves()
        
        # Check if there's a winning move for the agent
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.player
            if env.check_winner(move, self.player):
                return move  # Play the winning move

        # Check if the opponent has a winning move to block
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.opponent
            if env.check_winner(move, self.opponent):
                return move  # Block the opponent

        # Look for a move that could lead to a win in the future
        for move in available_moves:
            new_board = list(board)
            new_board[move] = self.player
            if any(env.check_winner(future_move, self.player) for future_move in env.available_moves()):
                return move

        # If no strategic moves, play a random move
        return random.choice(available_moves)

In [24]:

class QLearningAgent:
    def __init__(self, player, alpha=0.1, gamma=0.9, epsilon=1.0, epsilon_decay=0.99):
        self.q_table = {}  # Dictionnaire pour stocker Q-values
        self.alpha = alpha  # Taux d'apprentissage
        self.gamma = gamma  # Facteur de réduction
        self.epsilon = epsilon  # Facteur d'exploration
        self.epsilon_decay = epsilon_decay  # Décroissance de l'exploration

    def get_q_value(self, state, action):
        # Récupère la Q-value pour une action donnée dans un état donné
        return self.q_table.get((state, action), 0.0)

    
    def choose_action(self, state, available_actions):
        # Choisir une action selon la politique ε-greedy
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(available_actions)  # Exploration
        else:
            # Exploitation: choisir l'action avec la plus grande Q-value
            q_values = [self.get_q_value(state, action) for action in available_actions]
            max_q_value = max(q_values)
            max_q_actions = [action for action in available_actions if self.get_q_value(state, action) == max_q_value]
            return random.choice(max_q_actions)

    def update_q_table(self, state, action, reward, next_state, done):
        # Mise à jour de la Q-value
        max_future_q = max([self.get_q_value(next_state, a) for a in range(9)], default=0)
        current_q = self.get_q_value(state, action)
        if done:
            new_q = reward
        else:
            new_q = current_q + self.alpha * (reward + self.gamma * max_future_q - current_q)
        self.q_table[(state, action)] = new_q

    def decay_epsilon(self):
        self.epsilon *= self.epsilon_decay


In [7]:
env = TicTacToeEnv()

In [8]:
agent_First = QLearningAgent()
Smart_Agent = PreProgrammedAgent()


In [22]:
trainAiVsPreProgrammed(agent_First,Smart_Agent, env, 10000)


Episode 1000 - Wins Q-learning: 605, Wins PreProgrammed: 318, Draws: 77
Episode 2000 - Wins Q-learning: 640, Wins PreProgrammed: 305, Draws: 55
Episode 3000 - Wins Q-learning: 637, Wins PreProgrammed: 293, Draws: 70
Episode 4000 - Wins Q-learning: 636, Wins PreProgrammed: 291, Draws: 73
Episode 5000 - Wins Q-learning: 616, Wins PreProgrammed: 325, Draws: 59
Episode 6000 - Wins Q-learning: 620, Wins PreProgrammed: 300, Draws: 80
Episode 7000 - Wins Q-learning: 635, Wins PreProgrammed: 297, Draws: 68
Episode 8000 - Wins Q-learning: 626, Wins PreProgrammed: 316, Draws: 58
Episode 9000 - Wins Q-learning: 617, Wins PreProgrammed: 309, Draws: 74
Episode 10000 - Wins Q-learning: 637, Wins PreProgrammed: 295, Draws: 68
Training complete


In [21]:
trainAiVsRandom(agent_First, env, 100000, False)
trainAiVsRandom(agent_First, env, 100000, True)


Épisode 10000 - Victoires de l'IA: 5772, Défaites de l'IA: 3572, Égalités: 656
Épisode 20000 - Victoires de l'IA: 5735, Défaites de l'IA: 3688, Égalités: 577
Épisode 30000 - Victoires de l'IA: 5788, Défaites de l'IA: 3615, Égalités: 597
Épisode 40000 - Victoires de l'IA: 5666, Défaites de l'IA: 3729, Égalités: 605
Épisode 50000 - Victoires de l'IA: 5780, Défaites de l'IA: 3597, Égalités: 623
Épisode 60000 - Victoires de l'IA: 5745, Défaites de l'IA: 3670, Égalités: 585
Épisode 70000 - Victoires de l'IA: 5728, Défaites de l'IA: 3630, Égalités: 642
Épisode 80000 - Victoires de l'IA: 5742, Défaites de l'IA: 3636, Égalités: 622
Épisode 90000 - Victoires de l'IA: 5746, Défaites de l'IA: 3602, Égalités: 652
Épisode 100000 - Victoires de l'IA: 5766, Défaites de l'IA: 3603, Égalités: 631
Entraînement terminé
Épisode 10000 - Victoires de l'IA: 6821, Défaites de l'IA: 2496, Égalités: 683
Épisode 20000 - Victoires de l'IA: 6776, Défaites de l'IA: 2555, Égalités: 669
Épisode 30000 - Victoires de l

In [23]:
trainAivsAi(agent_First, agent_First, env)


Épisode 1000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 2000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 3000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 4000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 5000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 6000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 7000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 8000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 9000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Épisode 10000 - Victoires Agent1: 0, Victoires Agent2: 1000, Égalités: 0
Entraînement terminé
